# Project Description

# Set Up

## Check GPU

In [ ]:
import tensorflow as tf
from tensorflow.python.client import device_lib 
tf.__version__

In [ ]:
tf.config.list_physical_devices('GPU')

## Define Helper Functions

In [ ]:
def plot_loss_acc(history):
    '''
    Plots the training and validation accuracy and loss curves for fitted keras models
    In: The history object of a trained keras model
    Out: 2 Plots (Accuracy & Loss)
    '''
    fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(15,5))  
    fig.suptitle('Model Performance')

    # summarize history for accuracy
    ax1.plot(history.history['accuracy'])
    ax1.plot(history.history['val_accuracy'])
    ax1.set_title('Training and Validation Accuracy')
    ax1.set(xlabel='Epoch', ylabel='Accuracy')    
    ax1.legend(['Training Accuracy', 'Validation Accuracy'], loc='best')
    
    # summarize history for loss
    ax2.plot(history.history['loss'])
    ax2.plot(history.history['val_loss'])
    ax2.set_title('Training and Validation Loss')
    ax2.set(xlabel='Epoch', ylabel='Loss') 
    ax2.legend(['Training Loss', 'Validation Loss'], loc='best')
    
    return fig

def plot_conf_mat(model, load=False, load_loc=None, class_lst = 'Angry Disgust Fear Happy Sad Surprise Neutral', out='model_weights/finalconf.png'):
    import seaborn as sns
    import pandas as pd
    import matplotlib.pyplot as plt
    import math
    # 0=angry, 1=disgust,2=fear,3=happy, 4=sad, 5=surprise, 6=neutral
    classes = class_lst.split(' ')
    
    number_of_examples = len(validation_generator.filenames)
    number_of_generator_calls = math.ceil(number_of_examples / (1.0 * batch_size)) 
    # 1.0 above is to skip integer division
    y_true = []
    for i in range(0,int(number_of_generator_calls)):
        y_true.extend(np.array(validation_generator[i][1]))
        
    print(len(y_true))
    print('ones', sum(y_true))
    #y_true=[np.argmax(x) for x in val_data[1]]

    if load:
        model.load_weights(load_loc)
        mod_name = load_loc.split('-')[0]
    else:
        mod_name=out

        
    val_data = []
    for i in range(0,int(number_of_generator_calls)):
        val_data.extend(np.array(validation_generator[i][0]))
    
    val_data = np.array(val_data)
    
    print('Prediction Time')
    y_pred=np.argmax(model.predict(val_data),axis=1)
    
    
    con_mat = tf.math.confusion_matrix(labels=y_true, predictions=y_pred).numpy()

    con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)

    con_mat_df = pd.DataFrame(con_mat_norm,
                         index = classes, 
                         columns = classes)

    figure = plt.figure(figsize=(8, 8))
    ax = sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.BuGn)
    ax.set_ylim(len(classes),0)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.gcf().subplots_adjust(bottom=0.15, left=0.15)
    plt.savefig(mod_name+'_performance.png')
    return figure

# Build Models

## Preprocessing

## Package Imports

In [ ]:
from scipy.io import loadmat
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, Dropout
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

%matplotlib inline

## Parameter Set Up

In [ ]:
# dimensions of our images.
image_width, image_height = 128, 128
image_size=(image_width, image_height)
num_classes = 2

num_epochs = 100
batch_size = 64

dataset_name = 'gender'

patience = 20
input_shape = (image_width, image_height, 3)
if input_shape[2] == 1:
    grayscale = True

## Import Data

In [ ]:
train_data_dir = '../data/gender/train/'
validation_data_dir = '../data/gender/test/'

In [ ]:
def count_num_files(root=None):
    import os
    count=0
    for path, subdirs, files in os.walk(root):
        for name in files:
            count+=1
        
    return count

nb_train_samples = count_num_files('data/dataset/gender/train/')
nb_validation_samples = count_num_files('data/dataset/gender/test/')


In [ ]:
#ImageDataGenerator?

## MODEL: Simple CNN

In [ ]:
# simple cnn
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(7, 7), padding='same',
                        name='image_array', input_shape=input_shape))
model.add(BatchNormalization())
model.add(Conv2D(filters=16, kernel_size=(7, 7), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.5))

model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.5))

model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.5))

model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.5))

model.add(Conv2D(filters=256, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(
    filters=num_classes, kernel_size=(3, 3), padding='same'))
model.add(GlobalAveragePooling2D())
model.add(Activation('softmax', name='predictions'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# simple callbacks
log_file_path = 'logs/' + dataset_name + '_training.log'
csv_logger = CSVLogger(log_file_path, append=False)

early_stop = EarlyStopping('val_loss', patience=patience)

reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1, patience=int(patience/4), verbose=1)

trained_models_path = 'model_weights/' + dataset_name + '_simple_CNN'
model_names = trained_models_path + '.{epoch:02d}-{val_accuracy:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, monitor='val_accuracy', verbose=1, save_best_only=True)

callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]

In [ ]:
# fig = plot_conf_mat(model, load=True, load_loc='gender_models_trained/simple_CNN.81-0.96.hdf5', 
#               class_lst = 'Female Male', 
#               out='model_weights/finalconf_simplecnn.png')
# fig.show()

In [ ]:
history_simpleCNN = model.fit(train_generator, steps_per_epoch=nb_train_samples // batch_size, 
                                 epochs=num_epochs, validation_data=validation_generator, 
                                 validation_steps=nb_validation_samples // batch_size, verbose=1,
                                 callbacks=callbacks)

In [ ]:
fig = plot_loss_acc(history_simpleCNN)
fig.savefig('simpleCNN.gender.losscurve.png')
fig.show()

## MODEL: mini XCEPTION

In [ ]:
# mini XCEPTION callbacks
log_file_path = '../models/logs/' + dataset_name + '_training.log'
csv_logger = CSVLogger(log_file_path, append=False)

early_stop = EarlyStopping('val_loss', patience=patience)

reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1, patience=int(patience/4), verbose=1)

trained_models_path = '../models/weights/gender/' + dataset_name + '_mini_XCEPTION'
model_names = trained_models_path + '.{epoch:02d}-{val_accuracy:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, monitor='val_loss', verbose=1, save_best_only=True)

callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]

In [ ]:
#mini_XCEPTION
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import SeparableConv2D, MaxPooling2D
from tensorflow.keras import layers
from tensorflow.keras.models import Model

l2_regularization=0.01

regularization = l2(l2_regularization)

# base
img_input = Input(input_shape)
x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization,
           use_bias=False)(img_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization,
           use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

# module 1
residual = Conv2D(16, (1, 1), strides=(2, 2),
                  padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

x = SeparableConv2D(16, (3, 3), padding='same',
                    kernel_regularizer=regularization,
                    use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(16, (3, 3), padding='same',
                    kernel_regularizer=regularization,
                    use_bias=False)(x)
x = BatchNormalization()(x)

x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])

# module 2
residual = Conv2D(32, (1, 1), strides=(2, 2),
                  padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

x = SeparableConv2D(32, (3, 3), padding='same',
                    kernel_regularizer=regularization,
                    use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(32, (3, 3), padding='same',
                    kernel_regularizer=regularization,
                    use_bias=False)(x)
x = BatchNormalization()(x)

x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])

# module 3
residual = Conv2D(64, (1, 1), strides=(2, 2),
                  padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

x = SeparableConv2D(64, (3, 3), padding='same',
                    kernel_regularizer=regularization,
                    use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(64, (3, 3), padding='same',
                    kernel_regularizer=regularization,
                    use_bias=False)(x)
x = BatchNormalization()(x)

x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])

# module 4
residual = Conv2D(128, (1, 1), strides=(2, 2),
                  padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

x = SeparableConv2D(128, (3, 3), padding='same',
                    kernel_regularizer=regularization,
                    use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(128, (3, 3), padding='same',
                    kernel_regularizer=regularization,
                    use_bias=False)(x)
x = BatchNormalization()(x)

x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])

x = Conv2D(num_classes, (3, 3),
           # kernel_regularizer=regularization,
           padding='same')(x)
x = GlobalAveragePooling2D()(x)
output = Activation('softmax', name='predictions')(x)

model = Model(img_input, output)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history_mini_Xception = model.fit(train_generator, steps_per_epoch=nb_train_samples // batch_size, 
                                 epochs=num_epochs, validation_data=validation_generator, 
                                 validation_steps=nb_validation_samples // batch_size, verbose=1,
                                 callbacks=callbacks)

In [ ]:
# fig = plot_conf_mat(model, load=True, load_loc='gender_models_trained/gender_mini_XCEPTION.21-0.95.hdf5', 
#               class_lst = 'Female Male', 
#               out='model_weights/finalconf_genderxecption.png')
# fig.show()

In [ ]:
fig = plot_loss_acc(history_mini_Xception)
fig.show()

## MODEL: ResNet50 Finetuned

In [ ]:
train_data_dir = 'data/gender/train/'
validation_data_dir = 'data/gender/test/'
nb_train_samples = 171098 #count_num_files('data/dataset/gender/train/')
nb_validation_samples = 53742 #count_num_files('data/dataset/gender/test/')

In [ ]:
# dimensions of our images.
image_size=(64, 64)
image_width, image_height = 64, 64
num_classes = 2

num_epochs = 100
batch_size = 64

dataset_name = 'imdb'
validation_split = .1

# if K.image_data_format() == 'channels_first':
#     input_shape = (1, image_width, image_height)
# else:
#     input_shape = (image_width, image_height, 1)


do_random_crop = False
patience = 100
dataset_name = 'imdb'
input_shape = (64, 64, 3)
if input_shape[2] == 1:
    grayscale = True
    
images_path = 'data/imdb_crop/'

In [ ]:
#import tensorflow_hub as hub
print("TF version:", tf.__version__)
#print("Hub version:", hub.__version__)

In [ ]:
from scipy.io import loadmat
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, Dropout
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [ ]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
                        rescale=1. / 255,
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        #width_shift_range=0.1,
                        #height_shift_range=0.1,
                        #zoom_range=.1,
                        horizontal_flip=True,
                        preprocessing_function = preprocess_input)
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(image_width, image_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(image_width, image_height),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
top_model_weights_path = '../bottleneck_fc_model.h5'

#from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet_v2 import ResNet50V2 
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input
import numpy as np
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, Dropout
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


# build the resnet50 network
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(64,64,3))
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(512, activation='relu'))
top_model.add(Dropout(0.3))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.3))
top_model.add(Dense(128, activation='relu'))
top_model.add(Dropout(0.3))
top_model.add(Dense(1, activation='sigmoid'))

print(len(base_model.layers))
for layer in base_model.layers:
    layer.trainable = False

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
#top_model.load_weights(top_model_weights_path)

In [ ]:
# resnet summary
# base_model.summary()
# i=0
# for layer in base_model.layers:
#     layer.trainable = False
#     i = i+1
#     print(i,layer.name)

In [ ]:

# other models use this
# model.compile(optimizer='adam', loss='categorical_crossentropy',
#               metrics=['accuracy'])
model = Sequential()
model.add(base_model)
# add the model on top of the convolutional base
model.add(top_model)


# set the first 170 layers (up to the last conv block)
# to non-trainable (weights will not be updated)


    
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=5e-4, momentum=0.9),
              metrics=['accuracy'])

In [ ]:
#callbacks
log_file_path = 'logs/' + dataset_name + '_training.log'
csv_logger = CSVLogger(log_file_path, append=False)

early_stop = EarlyStopping('val_loss', patience=patience)

reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1, patience=int(patience/4), verbose=1)

trained_models_path = 'model_weights_gender_resnet/' + dataset_name + '_resnet50_finetune'
model_names = trained_models_path + '.{epoch:02d}-{val_accuracy:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, monitor='val_accuracy', verbose=1, save_best_only=True)

callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]

In [ ]:
# model.trainable=True

len(model.trainable_weights)


In [ ]:
#fit
resnet_finetune = model.fit(train_generator, steps_per_epoch=nb_train_samples // batch_size, 
                                 epochs=num_epochs, validation_data=validation_generator, 
                                 validation_steps=nb_validation_samples // batch_size, verbose=1,
                                 callbacks=callbacks)

In [ ]:
# resnet sample code to test
# img_path = '1280px-African_Bush_Elephant.jpg'
# img = image.load_img(img_path, target_size=(224, 224))
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)

# preds = model.predict(x)
# # decode the results into a list of tuples (class, description, probability)
# # (one such list for each sample in the batch)
# print('Predicted:', decode_predictions(preds, top=3)[0])
# # Predicted: [(u'n02504013', u'Indian_elephant', 0.82658225), (u'n01871265', u'tusker', 0.1122357), (u'n02504458', u'African_elephant', 0.061040461)]

## MODEL: ResNet50 Bottlenecked

In [ ]:
!pip install "tensorflow_hub>=0.6.0"

In [ ]:
#import tensorflow_hub as hub

print("TF version:", tf.__version__)
#print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

In [ ]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
                        rescale=1. / 255,
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        #width_shift_range=0.1,
                        #height_shift_range=0.1,
                        #zoom_range=.1,
                        horizontal_flip=True,
                        preprocessing_function = preprocess_input)
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(image_width, image_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(image_width, image_height),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
top_model_weights_path = '../bottleneck_fc_model.h5'

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dropout, Flatten, Dense


# build the resnet50 network
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(64,64,3))
print('Model loaded.')


In [ ]:
bottleneck_features_train = base_model.predict(
    train_generator, steps=nb_train_samples // batch_size)
#np.save(open('bottleneck_features_train.npy', 'wb'), bottleneck_features_train)



In [ ]:
bottleneck_features_validation = base_model.predict(
    validation_generator, steps=nb_validation_samples // batch_size)
#np.save(open('bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)

In [ ]:
# build a classifier model to put on top of the convolutional model
model = Sequential()
model.add(Flatten(input_shape=base_model.output_shape[1:]))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))



In [ ]:
# resnet summary
# base_model.summary()
# i=0
# for layer in base_model.layers:
#     layer.trainable = False
#     i = i+1
#     print(i,layer.name)

In [ ]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=5e-4, momentum=0.9),
              metrics=['accuracy'])

In [ ]:
#callbacks
log_file_path = 'logs/' + dataset_name + '_training.log'
csv_logger = CSVLogger(log_file_path, append=False)

early_stop = EarlyStopping('val_loss', patience=patience)

reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1, patience=int(patience/4), verbose=1)

trained_models_path = 'model_weights_gender_resnet/' + dataset_name + '_resnet50_finetune'
model_names = trained_models_path + '.{epoch:02d}-{val_accuracy:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, monitor='val_accuracy', verbose=1, save_best_only=True)

callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]

In [ ]:
# model.trainable=True

len(model.trainable_weights)


In [ ]:
#fit
resnet_finetune = model.fit(train_generator, steps_per_epoch=nb_train_samples // batch_size, 
                                 epochs=num_epochs, validation_data=validation_generator, 
                                 validation_steps=nb_validation_samples // batch_size, verbose=1,
                                 callbacks=callbacks)